In [1]:
import torch
from ultralytics import YOLO
import os
import yaml
import matplotlib.pyplot as plt
import cv2
from IPython.display import Image, display

# YOLOv8 uses the 'cuda' device automatically if available
print(f"Torch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

BASE_DIR = os.getcwd()
print(f"Project Directory: {BASE_DIR}")

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\comra\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Torch Version: 2.5.1+cu121
CUDA Available: True
Project Directory: C:\Users\comra\Dev\Aerial-Object-Detection


In [4]:
# Define Dataset Paths
dataset_path = os.path.join(BASE_DIR, 'data\object_detection_Dataset') 

# Define the dictionary structure for YOLO
data_config = {
    'path': dataset_path,          # Root dir
    'train': 'train/images',       # Train images (relative to 'path')
    'val': 'valid/images',         # Validation images (relative to 'path')
    'test': 'test/images',         # Test images (optional)
    
    # Class Names
    'nc': 2,                       # Number of Classes
    'names': ['Bird', 'Drone']     # Class names 
}

# Write this dictionary to a .yaml file
yaml_path = os.path.join(BASE_DIR, 'data.yaml')

with open(yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"Created Configuration File at: {yaml_path}")
print("Content:")
print(yaml.dump(data_config))

Created Configuration File at: C:\Users\comra\Dev\Aerial-Object-Detection\data.yaml
Content:
names:
- Bird
- Drone
nc: 2
path: C:\Users\comra\Dev\Aerial-Object-Detection\data\object_detection_Dataset
test: test/images
train: train/images
val: valid/images



In [5]:
# Initialize Model
# Load a pre-trained YOLOv8n model
model = YOLO('yolov8n.pt') 

print("Model Loaded: YOLOv8n")

# Start Training
# epochs=20: Sufficient for YOLO to learn simple objects like drones
# imgsz=640: Standard resolution for YOLO
# plots=True: Automatically saves graphs of loss/accuracy
results = model.train(
    data=yaml_path, 
    epochs=20, 
    imgsz=640, 
    batch=16,
    name='drone_bird_yolo', # Name of the save folder
    device=0 if torch.cuda.is_available() else 'cpu'
)

Model Loaded: YOLOv8n
Ultralytics 8.3.229  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\comra\Dev\Aerial-Object-Detection\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=drone_bird_yolo, nbs=64, nms=False, opset=None, optimize=False, opt

In [6]:
# Validate the Best Model
# YOLO automatically saves the best model to 'runs/detect/drone_bird_yolo/weights/best.pt'
best_model_path = os.path.join(BASE_DIR, 'runs', 'detect', 'drone_bird_yolo', 'weights', 'best.pt')

# Load the trained model
best_model = YOLO(best_model_path)

# Run validation metrics
metrics = best_model.val()

print(f"mAP50 (Accuracy at 50% overlap): {metrics.box.map50:.4f}")
print(f"mAP50-95 (Strict Accuracy):       {metrics.box.map:.4f}")

Ultralytics 8.3.229  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.30.2 ms, read: 35.223.5 MB/s, size: 24.2 KB)
val: Scanning C:\Users\comra\Dev\Aerial-Object-Detection\data\object_detection_Dataset\valid\labels.cache... 448 images, 6 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 448/448  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 28/28 2.1it/s 13.6s0.4s
                   all        448        663      0.871      0.752      0.821      0.534
                  Bird        217        414      0.833      0.628      0.737      0.431
                 Drone        225        249      0.909      0.876      0.905      0.637
Speed: 4.6ms preprocess, 6.6ms inference, 0.0ms loss, 6.1ms postprocess per image
Results saved to C:\Users\comra\Dev\Aerial-Object-Detection\runs\detect

In [7]:
import glob
import random

# Run Inference on Test Images
test_images_path = os.path.join(dataset_path, 'test', 'images', '*.jpg')
test_files = glob.glob(test_images_path)

# Pick 3 random images
random_files = random.sample(test_files, 3)

print("Running Inference on Test Data...")

plt.figure(figsize=(15, 5))

for i, file in enumerate(random_files):
    # Run prediction
    # conf=0.5 means "Only show boxes if you are 50% sure"
    results = best_model.predict(source=file, conf=0.5, save=False)
    
    # Plot the result
    res_plotted = results[0].plot() # Draws boxes on the image
    
    # Convert Color (OpenCV is BGR, Matplotlib is RGB)
    res_rgb = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)
    
    plt.subplot(1, 3, i+1)
    plt.imshow(res_rgb)
    plt.axis('off')
    plt.title(f"Test Image {i+1}")

plt.show()

Running Inference on Test Data...

image 1/1 C:\Users\comra\Dev\Aerial-Object-Detection\data\object_detection_Dataset\test\images\104f1bdceabcf7ba_jpg.rf.0b9745f785acd2ddb1dc562b1403efb3.jpg: 640x640 1 Bird, 94.7ms
Speed: 11.9ms preprocess, 94.7ms inference, 19.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\comra\Dev\Aerial-Object-Detection\data\object_detection_Dataset\test\images\06b3bc9d00e0bd0c_jpg.rf.db81bbe2cdd9edb36e3405001602520c.jpg: 640x640 1 Bird, 69.3ms
Speed: 11.0ms preprocess, 69.3ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\comra\Dev\Aerial-Object-Detection\data\object_detection_Dataset\test\images\pic_669_jpg.rf.04ceaf480d5110a1b1a0b625a42d7178.jpg: 640x640 1 Bird, 1 Drone, 38.0ms
Speed: 12.3ms preprocess, 38.0ms inference, 11.4ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 1500x500 with 3 Axes>